Esse notebook automatiza a conversão de um lote de arquivos PDF localizados em um diretório específico em imagens JPG.

Ele utiliza threads para um processamento mais rápido e inclui um mecanismo para retomar a conversão em caso de interrupções.
Para cada imagem processada, os metadados são lidos de um arquivo CSV (`selected_records_with_downloaded_pdfs.csv`), e um novo arquivo CSV (`records_with_images_metadata.csv`) é criado contendo campos de metadados específicos.
    
### 1. Importação de Bibliotecas Essenciais
Esta seção importa todas as bibliotecas necessárias para as operações deste notebook.

Elas são organizadas para facilitar a compreensão e manutenção.

In [ ]:
import os
import traceback
import threading 
from queue import Queue 
import logging           
from datetime import datetime
from typing import List, Dict, Tuple, Optional
import concurrent.futures  
import pandas as pd       
from pdf2image import convert_from_path
from tqdm.notebook import tqdm

### 2. Configuração de variáveis
Esta seção define as principais variáveis que controlam o comportamento do script de conversão de PDF para imagem e extração de metadados. 

**Você pode ajustar esses valores para personalizar a operação de acordo com suas necessidades.**

In [ ]:
# Diretório onde os arquivos PDF a serem convertidos estão localizados.
PDF_DIR: str = "PDFs"

# Diretório onde as imagens JPG resultantes da conversão serão salvas.
JPG_OUTPUT_DIR: str = "JPG_Images"

# Nome do arquivo CSV de entrada que contém os metadados originais e os caminhos dos PDFs baixados.
# Este é o CSV atualizado do notebook anterior.
RECORDS_CSV: str = "selected_records_with_downloaded_pdfs.csv"

# Nome do arquivo CSV de saída onde os metadados das imagens processadas serão registrados.
# Este arquivo será criado ou atualizado durante o processamento.
METADATA_CSV: str = "records_with_images_metadata.csv"

In [ ]:
# Número de threads (processos concorrentes) a serem usados para converter PDFs em imagens.
# Um número maior pode acelerar a conversão, mas consome mais recursos do sistema (CPU/RAM).
# Ajuste conforme a capacidade do seu hardware e a natureza dos PDFs.
NUM_WORKERS: int = 10

# Resolução (DPI - Dots Per Inch) para a conversão de PDF para imagem.
# Valores mais altos resultam em imagens de maior qualidade e clareza, mas também aumentam
# o tempo de processamento e o tamanho final dos arquivos JPG.
# Sugestões: 72 (tela), 150 (boa qualidade para visualização), 300 (qualidade de impressão).
DPI: int = 200

# Tamanho do "pedaço" (chunk) de PDFs a serem processados por cada thread.
# Isso afeta como as tarefas são distribuídas entre as threads.
# Um chunk_size muito pequeno pode aumentar o overhead; um muito grande pode desequilibrar a carga.
# Geralmente, um valor entre 10 e 50 é um bom ponto de partida.
CHUNK_SIZE: int = 10

In [ ]:
if not os.path.exists(JPG_OUTPUT_DIR):
    os.makedirs(JPG_OUTPUT_DIR)
    print(f"Diretório de saída para JPGs '{JPG_OUTPUT_DIR}' criado com sucesso.")
else:
    print(f"Diretório de saída para JPGs '{JPG_OUTPUT_DIR}' já existe.")

if not os.path.exists(PDF_DIR):
    print(f"AVISO: O diretório de PDFs de entrada '{PDF_DIR}' não foi encontrado.")
    print("Por favor, verifique se o caminho está correto e se os PDFs foram baixados para lá.")
else:
    print(f"Diretório de PDFs de entrada '{PDF_DIR}' verificado.")

### 3. Configuração do Sistema de Logging

Esta seção configura o sistema de log do Python para registrar o progresso das conversões, avisos e quaisquer erros em um arquivo dedicado (`pdf_conversion.log`). 

Isso é extremamente útil para monitorar a execução, depurar problemas e ter um histórico completo do processo, mesmo que o notebook seja fechado ou interrompido.

In [ ]:
LOG_FILE: str = "pdf_conversion.log"

logging.basicConfig(filename=LOG_FILE, level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(threadName)s - %(message)s')

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(threadName)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

if not any(isinstance(handler, logging.StreamHandler) for handler in logging.getLogger().handlers):
    logging.getLogger().addHandler(console_handler)

logging.info(f"Sistema de logging configurado. Logs serão gravados em '{LOG_FILE}'.")

### 4. Definindo Funções Auxiliares Essenciais
Esta seção define as funções de apoio que realizam tarefas específicas e repetitivas. 

#### 4.1. Verificação e Criação de Diretórios (ensure_directories_exist)

Garante que todos os diretórios necessários para salvar os arquivos de saída existam antes que o processo de conversão comece, prevenindo erros de "diretório não encontrado".

In [ ]:
def ensure_directories_exist(directories: List[str]) -> None:
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        logging.info(f"Diretório '{directory}' garantido (criado ou já existente).")

#### 4.2. Carregamento dos Dados do CSV (load_records_csv)

Esta função é responsável por carregar o arquivo CSV de entrada que contém os metadados.

Ela inclui tratamento de erros para lidar com arquivos ausentes, vazios ou colunas essenciais faltando, garantindo que o processamento só continue se os dados estiverem válidos.

In [ ]:
def load_records_csv(csv_path: str) -> Optional[pd.DataFrame]:
    try:
        df: pd.DataFrame = pd.read_csv(csv_path)
        logging.info(f"Arquivo CSV de registros '{csv_path}' carregado com sucesso. {len(df)} entradas encontradas.")

        required_columns = ['id', 'downloaded_pdf_filenames']
        for col in required_columns:
            if col not in df.columns:
                logging.error(f"Erro: Coluna essencial '{col}' não encontrada no arquivo CSV '{csv_path}'.")
                logging.error("Certifique-se de que o CSV é o resultado do primeiro notebook ou contém os caminhos dos PDFs.")
                return None

        df['id'] = df['id'].astype(str)
        
        df['downloaded_pdf_filenames'] = df['downloaded_pdf_filenames'].apply(
            lambda x: eval(x) if pd.notna(x) and isinstance(x, str) and x.strip().startswith('[') else []
        )
        logging.info("Coluna 'downloaded_pdf_filenames' convertida de string para lista de Python.")

        return df

    except FileNotFoundError:
        logging.error(f"Erro: Arquivo CSV de registros não encontrado em '{csv_path}'.")
        return None
    except pd.errors.EmptyDataError:
        logging.error(f"Erro: Arquivo CSV '{csv_path}' está vazio. Não há dados para processar.")
        return None
    except pd.errors.ParserError as e:
        logging.error(f"Erro de parsing no arquivo CSV '{csv_path}': {e}")
        logging.error(traceback.format_exc())
        return None
    except Exception as e:
        logging.error(f"Erro inesperado ao carregar o CSV '{csv_path}': {e}")
        logging.error(traceback.format_exc())
        return None

#### 4.3. Conversão de PDF para JPGs (pdf_to_jpgs)

Esta função utiliza a biblioteca `pdf2image` para converter um único arquivo PDF em uma ou mais imagens JPG (uma para cada página).

Ela retorna uma lista contendo o nome do arquivo JPG gerado e o número da página correspondente.

In [ ]:
def pdf_to_jpgs(pdf_path: str, output_dir: str, original_item_id: str, dpi: int) -> List[Tuple[str, int]]:
    jpg_data: List[Tuple[str, int]] = []
    try:
        images = convert_from_path(pdf_path, dpi=dpi)

        for i, image in enumerate(images):
            if len(images) == 1:
                jpg_filename = f"{original_item_id}.jpg"
            else:
                jpg_filename = f"{original_item_id}_page_{i + 1}.jpg"

            jpg_path = os.path.join(output_dir, jpg_filename)

            if not os.path.exists(jpg_path):
                image.save(jpg_path, 'JPEG')
                logging.info(f"[{threading.current_thread().name}] Gerado JPG: '{jpg_path}'.")
            else:
                logging.debug(f"[{threading.current_thread().name}] JPG '{jpg_path}' já existe. Pulando salvamento.")
            
            jpg_data.append((jpg_filename, i + 1))
        
        logging.info(f"[{threading.current_thread().name}] PDF '{os.path.basename(pdf_path)}' convertido para {len(images)} JPG(s).")
        return jpg_data

    except Exception as e:
        logging.error(f"[{threading.current_thread().name}] Erro ao converter PDF '{pdf_path}' para JPGs: {e}")
        logging.error(traceback.format_exc())
        return []

#### 4.4. Processamento de PDF Individual (process_pdf)

Esta é a função central que orquestra o processamento de um único arquivo PDF. Ela:

- Extrai o ID, verifica se o PDF já foi processado (para resumibilidade).
- Chama a função de conversão.
- Coleta os metadados correspondentes do DataFrame original.
- Enfileira os metadados para serem gravados no CSV de saída.

O uso de um Lock garante a segurança de threads ao atualizar o conjunto de IDs processados.

In [ ]:
def process_single_pdf_task(
    pdf_path: str,
    output_dir: str,
    records_df: pd.DataFrame,
    processed_ids_lock: threading.Lock,
    processed_ids_set: set,
    metadata_list_lock: threading.Lock,
    global_metadata_list: List[Dict],
    dpi: int
) -> None:
    
    thread_name = threading.current_thread().name
    logging.info(f"[{thread_name}] Iniciando processamento para PDF: '{os.path.basename(pdf_path)}'.")

    try:
        page_id_from_pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
        
        unique_download_id = page_id_from_pdf_name 

        logging.debug(f"[{thread_name}] PDF: '{os.path.basename(pdf_path)}' -> Page ID from filename (Unique Download ID): '{unique_download_id}'")

        with processed_ids_lock:
            if unique_download_id in processed_ids_set:
                logging.info(f"[{thread_name}] Pulando PDF já processado (pelo ID da página): '{os.path.basename(pdf_path)}'.")
                return
        
        original_item_id = None
        row_metadata = None

        pdf_filename_for_csv_check = os.path.join("PDFs", os.path.basename(pdf_path))
        
        matching_rows = records_df[
            records_df['downloaded_pdf_filenames'].apply(lambda x: pdf_filename_for_csv_check in x if isinstance(x, list) else False)
        ]

        if not matching_rows.empty:
            row_metadata = matching_rows.iloc[0]
            original_item_id = str(row_metadata['id']) # O ID do artigo
            logging.debug(f"[{thread_name}] Encontrado artigo pai '{original_item_id}' para PDF '{os.path.basename(pdf_path)}'.")
        else:
            logging.warning(f"[{thread_name}] Não foi possível encontrar o registro do artigo pai para o PDF '{os.path.basename(pdf_path)}' no 'records_df' usando a coluna 'downloaded_pdf_filenames'. Os metadados para este PDF não serão completos.")
            return 

        jpg_data: List[Tuple[str, int]] = pdf_to_jpgs(pdf_path, output_dir, page_id_from_pdf_name, dpi)
        
        if not jpg_data:
            logging.warning(f"[{thread_name}] Nenhuma imagem JPG gerada para o PDF: '{os.path.basename(pdf_path)}'. Pulando coleta de metadados.")
            return

        new_metadata_entries = []
        for jpg_filename, page_num in jpg_data:
            metadata_entry = {
                'original_article_id': original_item_id, 
                'page_id': page_id_from_pdf_name,        
                'download_id': unique_download_id,       
                'jpg_filename': jpg_filename,
                'jpg_page_number': page_num,         
                'trove_page_url': row_metadata.get('trove_page_url', 'N/A'), 
                'date': row_metadata.get('date', 'N/A'),
                'page': row_metadata.get('page', 'N/A'),
                'illustrated': row_metadata.get('illustrated', 'N/A'),
                'type': row_metadata.get('type', 'N/A'),
            }
            new_metadata_entries.append(metadata_entry)
        
        with metadata_list_lock:
            global_metadata_list.extend(new_metadata_entries)
        
        logging.info(f"[{thread_name}] {len(new_metadata_entries)} metadados para PDF '{os.path.basename(pdf_path)}' (Artigo: '{original_item_id}') coletados e enfileirados.")

        with processed_ids_lock:
            processed_ids_set.add(unique_download_id)
            logging.debug(f"[{thread_name}] ID único '{unique_download_id}' adicionado aos IDs processados.")

    except Exception as e:
        logging.error(f"[{thread_name}] Erro crítico ao processar PDF '{pdf_path}': {e}")
        logging.error(traceback.format_exc())

#### 4.7. Obtenção de IDs Já Processados (get_already_processed_ids)

Esta função é a base da capacidade de retomada do seu script. Ela:

1. Verifica se o arquivo CSV de metadados de saída já existe.
2. Se existir, ela lê a coluna 'id' desse arquivo e retorna um conjunto desses IDs.
3. Este conjunto é então usado para pular PDFs que já foram convertidos em execuções anteriores, economizando tempo e recursos.

In [ ]:
def get_already_processed_ids(metadata_file: str) -> set:
    processed_ids_set: set = set()
    if os.path.exists(metadata_file):
        try:
            existing_df: pd.DataFrame = pd.read_csv(metadata_file, dtype={'download_id': str})
            
            if 'download_id' in existing_df.columns:
                processed_ids_set = set(existing_df['download_id'].unique())
                logging.info(f"Encontrados {len(processed_ids_set)} IDs de downloads já processados em '{metadata_file}'.")
            else:
                logging.warning(f"Arquivo de metadados '{metadata_file}' não contém a coluna 'download_id'.")
                logging.warning("Todos os PDFs serão processados novamente para garantir a integridade.")
        except pd.errors.EmptyDataError:
            logging.info(f"Arquivo de metadados '{metadata_file}' está vazio. Nenhum ID processado encontrado.")
        except Exception as e:
            logging.warning(f"Erro ao ler IDs já processados do arquivo '{metadata_file}': {e}.")
            logging.warning("Todos os PDFs serão processados novamente para garantir a integridade.")
            logging.debug(traceback.format_exc())
    else:
        logging.info(f"Arquivo de metadados '{metadata_file}' não encontrado. Iniciando processamento do zero.")

    return processed_ids_set

### 5. Execução principal
Esta seção define a função principal, que orquestra todo o processo de conversão de PDF e extração de metadados.

Essa função principal configura as filas, os bloqueios e os threads necessários.
- Ela preenche a fila `pdf_queue` com pedaços de caminhos de arquivos PDF e inicia os threads de trabalho.

In [ ]:
def main(
    pdf_dir: str,
    jpg_output_dir: str,
    records_csv_path: str,
    metadata_csv_path: str,
    num_workers: int,
    dpi: int
) -> None:
    
    logging.info("--- INICIANDO PROCESSO PRINCIPAL DE CONVERSÃO DE PDF PARA JPG (concurrent.futures) ---")
    start_time = datetime.now()

    # 1. Garantir que os diretórios de saída existam
    ensure_directories_exist([jpg_output_dir])

    # 2. Carregar os metadados dos registros do CSV
    records_df = load_records_csv(records_csv_path)
    if records_df is None:
        logging.critical(f"Não foi possível carregar o arquivo CSV de registros '{records_csv_path}'. Encerrando.")
        return

    # 3. Obter IDs já processados (para funcionalidade de retomada)
    # processed_ids_set será um recurso compartilhado, acessado com lock.
    processed_ids_set = get_already_processed_ids(metadata_csv_path)
    logging.info(f"Retomada ativada: {len(processed_ids_set)} PDFs já processados serão pulados.")

     # 4. Preparar a lista de PDFs a serem processados
    pdf_paths_to_process: List[str] = []
    skipped_due_to_non_existence = 0
    skipped_due_to_already_processed = 0

    print("\nPreparando lista de PDFs elegíveis para conversão. Isso pode levar alguns segundos...")
    for _, row in tqdm(records_df.iterrows(), total=len(records_df), desc="Verificando PDFs do CSV"):
        if isinstance(row['downloaded_pdf_filenames'], list) and row['downloaded_pdf_filenames']:
            for pdf_filename_in_csv in row['downloaded_pdf_filenames']:
                full_pdf_path = os.path.join(pdf_dir, os.path.basename(pdf_filename_in_csv))

                unique_download_id = os.path.splitext(os.path.basename(full_pdf_path))[0]
                
                if os.path.exists(full_pdf_path):
                    if unique_download_id not in processed_ids_set:
                        pdf_paths_to_process.append(full_pdf_path)
                    else:
                        skipped_due_to_already_processed += 1
                        logging.debug(f"PDF '{os.path.basename(full_pdf_path)}' (ID da página: {unique_download_id}) já foi processado. Pulando.")
                else:
                    skipped_due_to_non_existence += 1
                    logging.warning(f"PDF '{os.path.basename(full_pdf_path)}' listado no CSV mas não encontrado em '{pdf_dir}'. Pulando.")
        else:
             logging.debug(f"Linha de registro com ID '{row['id']}' não tem 'downloaded_pdf_filenames' válido ou está vazia.")

    pdf_paths_to_process.sort()

    if not pdf_paths_to_process:
        logging.warning("Nenhum arquivo PDF novo ou não processado encontrado para conversão. Encerrando.")
        return

    logging.info(f"Total de {len(pdf_paths_to_process)} PDFs elegíveis para conversão.")

    global_metadata_list: List[Dict] = []
    processed_ids_lock = threading.Lock()
    metadata_list_lock = threading.Lock()

    # 5. Usar ThreadPoolExecutor para paralelizar a conversão
    print(f"\n**Iniciando extração de imagens de PDFs com {num_workers} workers (concurrent.futures)...**")
    logging.info(f"Iniciando ThreadPoolExecutor com {num_workers} workers.")

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(process_single_pdf_task, 
                                   pdf_path, 
                                   jpg_output_dir, 
                                   records_df, 
                                   processed_ids_lock, 
                                   processed_ids_set, 
                                   metadata_list_lock, 
                                   global_metadata_list, 
                                   dpi): pdf_path 
                   for pdf_path in pdf_paths_to_process}

        for future in tqdm(concurrent.futures.as_completed(futures), 
                           total=len(pdf_paths_to_process), 
                           desc="Progresso da Conversão JPG"):
            try:
                future.result() 
            except Exception as exc:
                pdf_path = futures[future]
                logging.error(f"'{pdf_path}' gerou uma exceção: {exc}")
                logging.error(traceback.format_exc())

    print("\n--- **Processo de Extração de Imagens Concluído!** ---")
    logging.info("Processo de extração de imagens concluído.")

    # 6. Salvar os metadados finais em CSV
    if not global_metadata_list:
        print("**Nenhuma imagem JPG foi gerada ou metadados foram coletados.**")
        logging.warning("Nenhuma imagem JPG gerada ou metadados coletados ao final do processo.")
    else:
        df_images_metadata = pd.DataFrame(global_metadata_list)
        print("\n**DataFrame de Metadados de Imagens Gerado:**")
        print(f"Total de {len(df_images_metadata)} registros de imagem com metadados.")
        display(df_images_metadata.head())

        df_images_metadata.to_csv(metadata_csv_path, index=False)
        print(f"\n**Metadados das imagens salvas em '{metadata_csv_path}'**")
        logging.info(f"Metadados das imagens salvas em '{metadata_csv_path}'.")

    # 7. Resumo Final
    end_time = datetime.now()
    duration = end_time - start_time
    logging.info("--- PROCESSO DE CONVERSÃO DE PDF PARA JPG FINALIZADO ---")
    logging.info(f"Tempo total de execução: {duration}")

    total_jpgs_generated = len([f for f in os.listdir(jpg_output_dir) if f.lower().endswith('.jpg')])
    logging.info(f"Total de arquivos JPG gerados/existentes em '{jpg_output_dir}': {total_jpgs_generated}")

    print("\nProcesso de conversão concluído. Verifique os logs para detalhes.")
    print(f"Arquivos JPG salvos em: {jpg_output_dir}")
    print(f"Metadados salvos em: {metadata_csv_path}")
    print("\n--- PROCESSAMENTO FINALIZADO ---")

### 6. Execução do Processo de Conversão

Esta seção demonstra como iniciar o processo de conversão de PDFs para JPGs e extração de metadados. 

Para executar, basta rodar a célula abaixo.

**Certifique-se** que: 
1. As bibliotecas pdf2image e pandas estão instaladas
```python
pip install pdf2image pandas
```
2. A biblioteca Poppler esteja instalada em seu sistema (consulte a documentação do pdf2image para obter instruções de instalação específicas para o seu sistema operacional).

In [ ]:
    main(
        pdf_dir=PDF_DIR,
        jpg_output_dir=JPG_OUTPUT_DIR,
        records_csv_path=RECORDS_CSV,
        metadata_csv_path=METADATA_CSV,
        num_workers=NUM_WORKERS,
        dpi=DPI
    )

### 7. Verificação de Integridade da Conversão

Esta função auxiliar é essencial para validar a qualidade e completude do processo de conversão. 

Ela verifica se todos os PDFs listados no seu CSV original que deveriam ter sido convertidos realmente têm entradas correspondentes no arquivo de metadados gerado. 

Além disso, ela identifica quaisquer entradas no CSV de metadados que não correspondem a um ID esperado do CSV de registros, ajudando a detectar inconsistências.

In [ ]:
def check_conversion_integrity(records_csv_path: str, metadata_csv_path: str) -> Tuple[set, List[Dict]]:
    expected_download_ids: set = set()
    try:
        records_df = pd.read_csv(records_csv_path)
        
        required_records_cols = ['id', 'downloaded_pdf_filenames']
        for col in required_records_cols:
            if col not in records_df.columns:
                logging.error(f"Erro: Coluna essencial '{col}' não encontrada em '{records_csv_path}'.")
                logging.error("Não é possível realizar a verificação de integridade sem as colunas corretas.")
                return set(), []

        records_df['id'] = records_df['id'].astype(str)
        records_df['downloaded_pdf_filenames'] = records_df['downloaded_pdf_filenames'].apply(
            lambda x: eval(x) if pd.notna(x) and x.strip().startswith('[') else []
        )

        for _, row in records_df.iterrows():
            for pdf_filename in row['downloaded_pdf_filenames']:
                unique_id = os.path.splitext(os.path.basename(pdf_filename))[0]
                expected_download_ids.add(unique_id)
        
        logging.info(f"Esperados {len(expected_download_ids)} IDs de downloads únicos com base em '{records_csv_path}'.")

    except FileNotFoundError:
        logging.error(f"Erro: Arquivo CSV de registros '{records_csv_path}' não encontrado. Verificação abortada.")
        return set(), []
    except pd.errors.EmptyDataError:
        logging.warning(f"Aviso: Arquivo CSV de registros '{records_csv_path}' está vazio. Nenhuma entrada esperada.")
        return set(), []
    except Exception as e:
        logging.error(f"Erro ao carregar ou processar '{records_csv_path}' para verificação: {e}")
        logging.error(traceback.format_exc())
        return set(), []

    found_download_ids: set = set()
    found_original_ids_in_metadata: set = set()
    metadata_df: Optional[pd.DataFrame] = None

    try:
        metadata_df = pd.read_csv(metadata_csv_path, dtype={'download_id': str, 'original_article_id': str})
        
        required_metadata_cols = ['download_id', 'original_article_id']
        for col in required_metadata_cols:
            if col not in metadata_df.columns:
                logging.error(f"Erro: Coluna essencial '{col}' não encontrada em '{metadata_csv_path}'.")
                logging.error("Não é possível realizar a verificação de integridade sem as colunas corretas.")
                return expected_download_ids, []

        found_download_ids = set(metadata_df['download_id'].unique())
        found_original_ids_in_metadata = set(metadata_df['original_article_id'].unique())

        logging.info(f"Encontrados {len(found_download_ids)} IDs de downloads únicos no CSV de metadados '{metadata_csv_path}'.")

    except FileNotFoundError:
        logging.warning(f"Aviso: Arquivo CSV de metadados '{metadata_csv_path}' não encontrado. Assumindo que todos os PDFs estão faltando.")
        return expected_download_ids, []
    except pd.errors.EmptyDataError:
        logging.warning(f"Aviso: Arquivo CSV de metadados '{metadata_csv_path}' está vazio. Nenhuma imagem foi processada.")
        return expected_download_ids, []
    except Exception as e:
        logging.error(f"Erro ao carregar ou processar '{metadata_csv_path}' para verificação: {e}")
        logging.error(traceback.format_exc())
        return expected_download_ids, []

    missing_download_ids = expected_download_ids - found_download_ids

    extra_metadata_entries: List[Dict] = []
    if records_df is not None and metadata_df is not None and not records_df.empty and not metadata_df.empty:
        valid_original_ids_from_records = set(records_df['id'].unique())
        
        extra_metadata_df = metadata_df[~metadata_df['original_article_id'].isin(valid_original_ids_from_records)]
        extra_metadata_entries = extra_metadata_df.to_dict('records')
        
        if extra_metadata_entries:
            logging.warning(f"Encontradas {len(extra_metadata_entries)} entradas 'extras' no CSV de metadados.")
        
    logging.info("Verificação de integridade concluída.")
    return missing_download_ids, extra_metadata_entries

Depois de rodar a célula principal (main_conversion_process), você pode rodar esta:

In [ ]:
missing_ids, extra_entries = check_conversion_integrity(RECORDS_CSV, METADATA_CSV)

if missing_ids:
    print(f"\n--- ATENÇÃO: {len(missing_ids)} PDFs não foram processados ---")
    print("IDs de downloads faltando:", missing_ids)
else:
    print("\nTodos os PDFs esperados foram encontrados no CSV de metadados.")

if extra_entries:
    print(f"\n--- ATENÇÃO: {len(extra_entries)} entradas 'extras' encontradas no CSV de metadados ---")
    print("Exemplos de entradas extras (primeiras 5):")
    for entry in extra_entries[:5]:
        print(entry)
else:
    print("\nNenhuma entrada 'extra' encontrada no CSV de metadados.")